In [30]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [41]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [ ]:
dividend_stars = ["KO", "JNJ", "PG", "PEP", "O", "XOM", "JPM", "MMM", "CVX", "ABBV"]

def fetch_financial_raw_data(tickers):
    results = []
    for ticker in tickers:
        print(f"Pobieram dane dla: {ticker}...")
        stock = yf.Ticker(ticker)
        try:
            inc = stock.income_stmt
            cf = stock.cash_flow
            bs = stock.balance_sheet
            
            # Pobieramy dywidendy na akcję do CAGR
            divs_per_share = stock.dividends
            yearly_divs_ps = divs_per_share.groupby(divs_per_share.index.year).sum()

            for report_date in inc.columns:
                year = report_date.year
                start_search = report_date - timedelta(days=5)
                hist_price = stock.history(start=start_search, end=report_date + timedelta(days=1))
                price_at_time = hist_price['Close'].iloc[-1] if not hist_price.empty else None

                # POBIERANIE DANYCH DO WSKAŹNIKÓW
                net_income = inc.loc['Net Income', report_date] if 'Net Income' in inc.index else None
                fcf = cf.loc['Free Cash Flow', report_date] if 'Free Cash Flow' in cf.index else None
                
                # Łączna kwota dywidend wypłacona przez firmę (zazwyczaj liczba ujemna w CF)
                total_div_paid = cf.loc['Cash Dividends Paid', report_date] if 'Cash Dividends Paid' in cf.index else None
                # Jeśli nie ma 'Cash Dividends Paid', szukamy 'Dividends Paid'
                if total_div_paid is None:
                    total_div_paid = cf.loc['Dividends Paid', report_date] if 'Dividends Paid' in cf.index else None

                results.append({
                    "Ticker": ticker,
                    "Rok": year,
                    "Net_Income": net_income,
                    "Free_Cash_Flow": fcf,
                    "Total_Div_Paid_Company": total_div_paid, # Łącznie (miliardy)
                    "Div_Per_Share": yearly_divs_ps.get(year, 0), # Na akcję (USD)
                    "Total_Debt": bs.loc['Total Debt', report_date] if 'Total Debt' in bs.index else None,
                    "Cash_And_Equiv": bs.loc['Cash And Cash Equivalents', report_date] if 'Cash And Cash Equivalents' in bs.index else 0,
                    "Price_At_Year_End": price_at_time
                })
        except Exception as e:
            print(f"Błąd przy {ticker}: {e}")
    return pd.DataFrame(results)

df_raw = fetch_financial_raw_data(dividend_stars)

# Ticker - symbol spółki
# Rok - rok raportu finansowego
# Net_Income - zysk netto (w USD)
# Free_Cash_Flow - wolne przepływy pieniężne (w USD)
# Total_Div_Paid_Company - łączna kwota dywidend wypłacona przez firmę (w USD)
# Div_Per_Share - dywidenda na akcję (w USD)
# Total_Debt - całkowite zadłużenie (w USD)
# Cash_And_Equiv - gotówka i ekwiwalenty (w USD)
# Price_At_Year_End - cena akcji na koniec roku (w USD)


Pobieram dane dla: KO...
Pobieram dane dla: JNJ...
Pobieram dane dla: PG...
Pobieram dane dla: PEP...
Pobieram dane dla: O...
Pobieram dane dla: XOM...
Pobieram dane dla: JPM...
Pobieram dane dla: MMM...
Pobieram dane dla: CVX...
Pobieram dane dla: ABBV...


In [49]:
df_raw.head(10)

,Ticker,Rok,Net_Income,Free_Cash_Flow,Total_Div_Paid_Company,Div_Per_Share,Total_Debt,Cash_And_Equiv,Price_At_Year_End
0,KO,2024,1.063100e+10,4.741000e+09,-8.359000e+09,1.940,4.452200e+10,1.082800e+10,60.475769
1,KO,2023,1.071400e+10,9.747000e+09,-7.952000e+09,1.840,4.206400e+10,9.366000e+09,55.544769
2,KO,2022,9.542000e+09,9.534000e+09,-7.616000e+09,1.760,3.914900e+10,9.519000e+09,58.122211
3,KO,2021,9.771000e+09,1.125800e+10,-7.252000e+09,1.680,4.276100e+10,9.684000e+09,52.547188
4,JNJ,2024,1.406600e+10,1.805900e+10,-1.182300e+10,4.910,3.663400e+10,2.410500e+10,140.324570
5,JNJ,2023,3.515300e+10,1.777800e+10,-1.177000e+10,4.700,2.933200e+10,2.185900e+10,147.417480
6,JNJ,2022,1.794100e+10,1.718500e+10,-1.168200e+10,4.450,3.964200e+10,1.288900e+10,161.250122
7,JNJ,2021,2.087800e+10,1.975800e+10,-1.103200e+10,4.190,3.375100e+10,1.448700e+10,152.158813
8,PG,2025,1.597400e+10,1.404400e+10,-9.872000e+09,4.178,3.546300e+10,9.556000e+09,157.139008
9,PG,2024,1.487900e+10,1.652400e+10,-9.312000e+09,3.962,3.336900e+10,9.482000e+09,158.780823


In [51]:
def calculate_ratios(df):
    # Tworzymy kopię, aby nie modyfikować oryginalnych danych
    metrics = df.copy()

    # 1. Dividend Payout Ratio (%) -> Czy zysk pokrywa dywidendę?
    # Używamy abs(), bo Total_Div_Paid_Company w raportach jest ujemne
    metrics['Payout_Ratio_NI_%'] = (abs(metrics['Total_Div_Paid_Company']) / metrics['Net_Income']) * 100

    # 2. FCF Payout Ratio (%) -> Czy gotówka pokrywa dywidendę? (Kluczowe!)
    metrics['Payout_Ratio_FCF_%'] = (abs(metrics['Total_Div_Paid_Company']) / metrics['Free_Cash_Flow']) * 100

    # 3. Net Debt (Dług netto) -> Ile długu zostanie po spłaceniu go całą gotówką
    metrics['Net_Debt'] = metrics['Total_Debt'] - metrics['Cash_And_Equiv']

    # 4. Dividend Yield (%) -> Historyczna stopa dywidendy w danym roku
    metrics['Yield_%'] = (metrics['Div_Per_Share'] / metrics['Price_At_Year_End']) * 100

    return metrics

df_with_metrics = calculate_ratios(df_raw)

In [53]:
df_with_metrics.head(10)

,Ticker,Rok,Net_Income,Free_Cash_Flow,Total_Div_Paid_Company,Div_Per_Share,Total_Debt,Cash_And_Equiv,Price_At_Year_End,Payout_Ratio_NI_%,Payout_Ratio_FCF_%,Net_Debt,Yield_%
0,KO,2024,1.063100e+10,4.741000e+09,-8.359000e+09,1.940,4.452200e+10,1.082800e+10,60.475769,78.628539,176.313014,3.369400e+10,3.207896
1,KO,2023,1.071400e+10,9.747000e+09,-7.952000e+09,1.840,4.206400e+10,9.366000e+09,55.544769,74.220646,81.584077,3.269800e+10,3.312643
2,KO,2022,9.542000e+09,9.534000e+09,-7.616000e+09,1.760,3.914900e+10,9.519000e+09,58.122211,79.815552,79.882526,2.963000e+10,3.028102
3,KO,2021,9.771000e+09,1.125800e+10,-7.252000e+09,1.680,4.276100e+10,9.684000e+09,52.547188,74.219630,64.416415,3.307700e+10,3.197126
4,JNJ,2024,1.406600e+10,1.805900e+10,-1.182300e+10,4.910,3.663400e+10,2.410500e+10,140.324570,84.053747,65.468741,1.252900e+10,3.499031
5,JNJ,2023,3.515300e+10,1.777800e+10,-1.177000e+10,4.700,2.933200e+10,2.185900e+10,147.417480,33.482206,66.205422,7.473000e+09,3.188224
6,JNJ,2022,1.794100e+10,1.718500e+10,-1.168200e+10,4.450,3.964200e+10,1.288900e+10,161.250122,65.113427,67.977888,2.675300e+10,2.759688
7,JNJ,2021,2.087800e+10,1.975800e+10,-1.103200e+10,4.190,3.375100e+10,1.448700e+10,152.158813,52.840310,55.835611,1.926400e+10,2.753702
8,PG,2025,1.597400e+10,1.404400e+10,-9.872000e+09,4.178,3.546300e+10,9.556000e+09,157.139008,61.800426,70.293364,2.590700e+10,2.658792
9,PG,2024,1.487900e+10,1.652400e+10,-9.312000e+09,3.962,3.336900e+10,9.482000e+09,158.780823,62.584851,56.354394,2.388700e+10,2.495264


In [ ]:
def calculate_cagr_3y(df):
    cagr_results = []
    
    # Grupowany po spółce, żeby liczyć wzrost dla każdego tickera osobno
    for ticker in df['Ticker'].unique():
        ticker_data = df[df['Ticker'] == ticker].sort_values('Rok')
        
        if len(ticker_data) >= 4:  # Potrzebujemy min. 4 lat, by mieć 3 pełne okresy wzrostu
            v_final = ticker_data['Div_Per_Share'].iloc[-1]
            v_start = ticker_data['Div_Per_Share'].iloc[0]
            
            if v_start > 0:
                cagr = ((v_final / v_start)**(1/3) - 1) * 100
            else:
                cagr = 0
                
            cagr_results.append({"Ticker": ticker, "3Y_Dividend_CAGR_%": round(cagr, 2)})
            
    return pd.DataFrame(cagr_results)

df_cagr_3y = calculate_cagr_3y(df_with_metrics)

# Pokazujemy wyniki CAGR - O ile średnio rosła dywidenda na akcję przez ostatnie 3 lata

In [55]:
df_cagr_3y

,Ticker,3Y_Dividend_CAGR_%
0,KO,4.91
1,JNJ,5.43
2,PG,5.00
3,PEP,7.86
4,O,1.15
5,XOM,3.24
6,JPM,7.53
7,MMM,-12.09
8,CVX,7.08
9,ABBV,6.04
